In [ ]:
contractions = { 
    "ain't": "am not / are not / is not / has not / have not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had / he would",
    "he'd've": "he would have",
    "he'll": "he shall / he will",
    "he'll've": "he shall have / he will have",
    "he's": "he has / he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how has / how is / how does",
    "I'd": "I had / I would",
    "I'd've": "I would have",
    "I'll": "I shall / I will",
    "I'll've": "I shall have / I will have",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it had / it would",
    "it'd've": "it would have",
    "it'll": "it shall / it will",
    "it'll've": "it shall have / it will have",
    "it's": "it has / it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she had / she would",
    "she'd've": "she would have",
    "she'll": "she shall / she will",
    "she'll've": "she shall have / she will have",
    "she's": "she has / she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as / so is",
    "that'd": "that would / that had",
    "that'd've": "that would have",
    "that's": "that has / that is",
    "there'd": "there had / there would",
    "there'd've": "there would have",
    "there's": "there has / there is",
    "they'd": "they had / they would",
    "they'd've": "they would have",
    "they'll": "they shall / they will",
    "they'll've": "they shall have / they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had / we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what shall / what will",
    "what'll've": "what shall have / what will have",
    "what're": "what are",
    "what's": "what has / what is",
    "what've": "what have",
    "when's": "when has / when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where has / where is",
    "where've": "where have",
    "who'll": "who shall / who will",
    "who'll've": "who shall have / who will have",
    "who's": "who has / who is",
    "who've": "who have",
    "why's": "why has / why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had / you would",
    "you'd've": "you would have",
    "you'll": "you shall / you will",
    "you'll've": "you shall have / you will have",
    "you're": "you are",
    "you've": "you have"
}

## Sources: <br>

text classification stuff: https://monkeylearn.com/text-classification/ <br>
Multinomial Naive Bayes:
* Really good results with a small dataset (couple thousand tagged samples) <br>
* Computationally efficient

Support Vector Machines:
* Like Naive Bayes, doesn't require much data for accurate results
* Faster and more accurate results, but more computationally demanding

Deep Learning:
* High accuracy with low-level engineering and computation
* Main models for text classifications are CNN and RNN
* Requires millions of tagged examples


NLP: 
* nltk provides 9 different stemming options
* scapy provides 1, but also has word embeddings


Evaluation:
* Recall: Rate of tweets in a class given the correct label
* Precision: Rate a label has been given to the correct class
* Accuracy: Rate of tweets given the correct label 
* F1-Score: The harmonic mean between recall and precision

<br>
<br>
<br>


to stem or to lem? https://towardsdatascience.com/stemming-vs-lemmatization-2daddabcb221 <br>
Stemming:
* Crude method for cataloging related words, reduces words down to a stem version
* Faster

Lemmatization:
* More informative, looks at surrounding text to determine the context of the word
* Slower


<br>
<br>
<br>

spacy vs nltk - https://medium.com/@akankshamalhotra24/introduction-to-libraries-of-nlp-in-python-nltk-vs-spacy-42d7b2f128f2 <br>
* Spacy uses latest and best algorithms, much faster word tokenization and part-of-speech tagging than nltk


<br>
<br>
<br>



Why and how to use BERT for text classification - https://www.analyticsvidhya.com/blog/2021/06/why-and-how-to-use-bert-for-nlp-text-classification/ <br>
* Normal language models reads left to right or right to left, Bert is bidirectional or non-directional and takes multiple tokens at the same time
* BERT is just an encoder which is responsible for reading text input and processing. The decoder is responsible for producing predictions.
* Multi-layered encoder, BERT base has 12 layers of encoders 


<br>
<br>
<br>

Other: <br>
https://www.analyticsvidhya.com/blog/2021/11/a-guide-to-automated-deep-machine-learning-for-natural-language-processing-text-prediction/


<br>
<br>
<br>
<hr>



Subtask A: <br>
Check-worthiness of tweets: Given a tweet, predict whether it is worth fact-checking. This task is defined with binary labels: Yes and No.. This is a classification task.
<br><br>
Subtask B: <br>
Verifiable factual claims detection: Given a tweet, predict whether it contains a verifiable factual claim. This is a binary task with two labels: Yes and No. This is a classification task
<br><br>
Subtask C: <br>
Harmful tweet detection : Given a tweet, predict whether it is harmful to the society and why. This task is defined with binary labels: Yes and No. This is a classification task.
<br><br>
Subtask D: <br>
Attention-worthy tweet detection: Given a tweet, predict whether it should get the attention of policy makers and why. This task is defined with eight class labels: (i) No, not interesting, (ii) not sure, (iii) Yes, asks question, (iv) Yes, blame authorities, (v) Yes, calls for action, (vi) Yes, classified as in harmful task, (vii) Yes, contains advice, (viii) Yes, discusses action taken, (ix) Yes, discusses cure, and (x) Yes, other. This is a classification task. 
<br><br>
Evaluation: <br>
This task is evaluated as a classification task. For the subtasks 1A and 1C, we use the F1 measure with respect to the positive class, for subtask 1B, we use accuracy, and for subtask 1D, we use weighted-F1

<br><br><br>

<hr>

## Loading Datasets & Plugins

In [556]:
import pandas as pd
import numpy as np
import json
import nltk
from nltk import ngrams
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import spacy

# Load Lemmatizer
nlp = spacy.load("en_core_web_sm")

# Load Stopwords
nltk.download('stopwords')
cachedStopWords = list(set(stopwords.words("english")))



# a)
# Twitter Datasets
df_worthy_train = pd.read_csv("checkworthy/CT22_english_1A_checkworthy_train.tsv", sep='\t')
df_worthy_test = pd.read_csv("checkworthy/CT22_english_1A_checkworthy_dev_test.tsv", sep='\t')

# Top 3000 most commonly used english words + covid related medical terms (used as vocabulary for the vectorization process)
topwords = pd.read_csv('words.txt', sep=" ", header=None).values[:,0]

[nltk_data] Downloading package stopwords to /Users/ivarw/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<br>

### Emoji Removal

In [930]:
# Function to remove emojies when using the lemmatizer
#https://poopcode.com/how-to-remove-emoji-from-text-in-python/
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

<br>

### Class Priors

In [929]:
samples = len(df_worthy_train)
class1 = (df_worthy_train["class_label"] == 0).sum()
class2 = (df_worthy_train["class_label"] == 1).sum()
print(f"Class 1 (Not Worthy): {round(class1/samples, 2)}")
print(f"Class 2 (Worthy): {round(class2/samples, 2)}")

Class 1 (Not Worthy): 0.79
Class 2 (Worthy): 0.21


<br><br><br>

## Custom Tokenizers

In [597]:
def tokenize_text(text, method="stopwords"):
    
    # Remove hyperlinks
    text = re.sub(r'http\S+', '', text)
    
    # Split sentence into words
    tokens = []
    for token in ngrams(text.split(), 1):
        word = re.sub(r',', '', token[0]) #Remove commas
        word = re.sub(r'[\!\.\:]$', '', word) #Remove (.!:) at the end of tokens
        word = re.sub(r'#', '', word) #Remove hashtags (not the text)
        if word == "—": continue #Ignore dash
        if word.find("@") != -1: continue #Ignore tags
        tokens.append(word)
    tokens

    # List comprehension to remove stopwords
    if method != "default":
        tokens = [x for x in tokens if x not in cachedStopWords]

    # Perform stemming
    if method == "stemming":
        ps = PorterStemmer()
        tokens = [ps.stem(x) for x in tokens]

    # Perform lemmatization
    if method == "lemmatizer":
        # Remove emojies due to lemmatizer not handling them well
        tokens = [x for x in tokens if remove_emoji(x) != ""]
        text = " ".join(tokens)
        doc = nlp(text)
        tokens = [token.lemma_ for token in doc]

    return tokens

In [747]:
def tokenize_normal(text):
    return tokenize_text(text, method="default")

def tokenize_stopwords(text):
    return tokenize_text(text, method="stopwords")

def tokenize_stemming(text):
    return tokenize_text(text, method="stemming")

def tokenize_lemmatizer(text):
    return tokenize_text(text, method="lemmatizer")

In [748]:
def tokenize_df(df, text_col, method="default"):
    article_tokens = []
    for i in range(len(df)):
        text = df.iloc[i][text_col].lower()
        article_tokens.append(tokenize_text(text, method))
    df["tokens"] = article_tokens

In [927]:
tokenize_df(df_worthy_train, "tweet_text", "lemmatizer")
tokenize_df(df_worthy_test, "tweet_text", "lemmatizer")

In [928]:
print(df_worthy_train["tokens"][0], "\n")
print(df_worthy_test["tokens"][0])

['india', "'s", 'gift', '100000', 'covid-19', 'vaccine', 'arrive', 'barbado', 'early', 'today', 'special', 'moment', 'barbadian', 'want', 'thank', 'prime', 'minister', 'modi', 'quick', 'decisive', 'magnanimous', 'action', 'allow', 'we', 'beneficiary', 'vaccine'] 

['senate', 'pass', 'covid', 'relief', '$', '1400', 'relief', 'check', 'fund', 'vaccine', 'money', 'reopen', 'school', 'food', 'unemployment', 'rental', 'assistance', 'cut', 'child', 'poverty', 'half', 'help', 'small', 'business', 'must', 'end', 'pandemic', 'help', 'way']


<br><br><br>

## Create Ngrams

In [756]:
def create_ngrams(df, n=1):
    # Don't execute code if no tokens column
    if "tokens" not in df.columns.tolist():
        print("The dataframe does not have a tokens columns")
        return

    # If monogram, copy token column to ngram column
    if n == 1:
        df["ngrams"] = df["tokens"]

    # Create ngrams
    article_ngrams = []
    for i in range(len(df)):
        ngrams = []
        tokens = df.iloc[i]["tokens"]
        for j in range(len(tokens)-n-1):
            ngrams.append(' '.join(tokens[j:j+n]))
        article_ngrams.append(ngrams)
    df["ngrams"] = article_ngrams
        

In [917]:
create_ngrams(df_worthy_train, n=1)
create_ngrams(df_worthy_test, n=1)

In [918]:
print(df_worthy_train.iloc[0]["ngrams"])
print()
print(df_worthy_test.iloc[0]["ngrams"])

["india's", 'gift', '100000', 'covid-19', 'vaccines', 'arrived', 'barbados', 'earlier', 'today', 'special', 'moment', 'barbadians', 'want', 'thank', 'prime', 'minister', 'modi', 'quick', 'decisive', 'magnanimous', 'action', 'allowing', 'us']

['senate', 'passed', 'covid', 'relief', '✔️', '$1400', 'relief', 'checks', '✔️', 'funding', 'vaccines', '✔️', 'money', 'reopen', 'schools', '✔️', 'food', 'unemployment', 'rental', 'assistance', '✔️', 'cutting', 'child', 'poverty', 'half', '✔️', 'help', 'small', 'businesses', 'must', 'end', 'pandemic']


<br><br><br>

## Bag of Words

In [981]:
from sklearn.feature_extraction.text import CountVectorizer # For creating a DTM (discrete values)
from sklearn.feature_extraction.text import TfidfVectorizer # A weighted DTM (fractions)

cv = CountVectorizer(tokenizer=tokenize_lemmatizer, ngram_range=(1, 1), max_features=5000) #Test with different n-grams, min_df, max_df and max_features
tfidf = TfidfVectorizer(tokenizer=tokenize_stopwords, ngram_range=(1, 1), max_features=5000) #Test with (1,1), (2,2), and (3,3) 

In [982]:
tweet_vec = cv.fit(df_worthy_train["tweet_text"]) # DTM (CV, but normalized for relative frequency)
tweet_bow_train = tweet_vec.transform(df_worthy_train["tweet_text"]) 
tweet_bow_test = tweet_vec.transform(df_worthy_test["tweet_text"]) 

/Users/ivarw/miniforge3/envs/DAT540/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


<br><br><br>

## Feature Exploration

In [969]:
# If max_features not limited, there are 8263 unique words (features) and a total of 51243 words across 2123 tweets
captured = tweet_bow_train.sum()
print(f"Total words across all tweets: 51243")
print(f"Total words captured by our features/vocabulary {captured} ({round(100*captured/51243, 1)}%)")
print(f"Amount of unique words (tokens/features): {len(tweet_vec.vocabulary_)}\n") # Unique words (features)

# vec.vocabulary_items() => Dict with every word in the vocabulary with the DTM column index as the value
sum_words = tweet_bow_train.sum(axis=0) # Sum the columns from the DTM matrix
words_freq = [(word, sum_words[0, index]) for word, index in tweet_vec.vocabulary_.items()] # Make a list pairing every word with its corresponding DTM column summation
words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
print(f"Most frequent terms: \n {words_freq[0:30]}") # show top n terms used

Total words across all tweets: 51243
Total words captured by our features/vocabulary 8256.0441105884 (16.1%)
Amount of unique words (tokens/features): 5000

Most frequent terms: 
 [('vaccines', 94.56203516442616), ('covid-19', 85.522268820762), ('vaccine', 73.72037894238488), ('covid', 37.707286575219364), ('people', 33.75806726936624), ('coronavirus', 31.909389236252974), ('get', 31.724904233717577), ('covid19', 30.51490446509189), ('health', 28.298205207340526), ('first', 27.8655325203803), ('&amp;', 26.31365473647958), ('doses', 25.22124108515064), ('new', 24.592987725151392), ('us', 24.2034157223064), ('effective', 21.567158227295884), ('countries', 21.456205995655537), ('one', 20.677083146734795), ('safe', 20.16660738412668), ('available', 19.48534754366807), ('today', 18.822161463391488), ('pandemic', 18.462544073969834), ('-', 18.24823760199171), ('world', 17.78358289232317), ('march', 17.4164450320283), ('virus', 17.346100667465485), ('johnson', 17.306375764729317), ('india', 1

<br><br><br>

## Dummy Predictions

In [970]:
from sklearn.dummy import DummyClassifier
from sklearn import metrics

dummy_clf = DummyClassifier(strategy="prior")
dummy_clf.fit(tweet_bow_train, df_worthy_train["class_label"])
print(f"Priors: {dummy_clf.class_prior_}")

accuracy_score = metrics.accuracy_score(dummy_clf.predict(tweet_bow_test), df_worthy_test["class_label"])
print(f"Dummy Accuracy: {round(accuracy_score * 100, 2)}%")

f1_score = metrics.f1_score(dummy_clf.predict(tweet_bow_test), df_worthy_test["class_label"])
print(f"Dummy F1-Score: {round(f1_score * 100, 2)}%")

Priors: [0.78934967 0.21065033]
Dummy Accuracy: 77.53%
Dummy F1-Score: 0.0%


<br><br><br>

## Naive Bayes Predictions

In [983]:
#https://scikit-learn.org/stable/modules/naive_bayes.html#naive-bayes
from sklearn.naive_bayes import MultinomialNB # suited for classification with discrete features
from sklearn.naive_bayes import ComplementNB # suited for imbalanced datasets (designed to correct for "severe assumptions")
from sklearn.naive_bayes import BernoulliNB # designed for classification with binary/boolean features
from sklearn.naive_bayes import GaussianNB # designed for classification with continuous values

clf = ComplementNB()
clf.fit(tweet_bow_train.toarray(), df_worthy_train["class_label"])

accuracy_score = metrics.accuracy_score(clf.predict(tweet_bow_test.toarray()), df_worthy_test["class_label"])
print(f"Complement Accuracy: {round(accuracy_score * 100, 2)}%")

accuracy_score = metrics.f1_score(clf.predict(tweet_bow_test.toarray()), df_worthy_test["class_label"])
print(f"Complement F1-Score: {round(accuracy_score * 100, 2)}%")

Complement Accuracy: 81.36%
Complement F1-Score: 59.62%


<br><br><br>

## Linear Support Vector Machine

In [916]:
#https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html

# based on liblinear
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

clf = make_pipeline(StandardScaler(), LinearSVC(random_state=0, tol=1e-5))
clf.fit(tweet_bow_train.toarray(), df_worthy_train["class_label"])

accuracy_score = metrics.accuracy_score(clf.predict(tweet_bow_test.toarray()), df_worthy_test["class_label"])
print(f"Linear SVC Accuracy: {round(accuracy_score * 100, 2)}%")

f1_score = metrics.f1_score(clf.predict(tweet_bow_test.toarray()), df_worthy_test["class_label"])
print(f"Linear SVC F1-Score: {round(f1_score * 100, 2)}%")

Linear SVC Accuracy: 77.7%
Linear SVC F1-Score: 47.97%


<br><br><br>

## C-Support Vector Predictions

In [986]:
# based on libsvm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

clf = make_pipeline(StandardScaler(), SVC(gamma='auto', ))
clf.fit(tweet_bow_train.toarray(), df_worthy_train["class_label"])

accuracy_score = metrics.accuracy_score(clf.predict(tweet_bow_test.toarray()), df_worthy_test["class_label"])
print(f"C-Support Vector Accuracy: {round(accuracy_score * 100, 2)}%")

f1_score = metrics.f1_score(clf.predict(tweet_bow_test.toarray()), df_worthy_test["class_label"])
print(f"C-Support Vector F1-Score: {round(f1_score * 100, 2)}%")

C-Support Vector Accuracy: 78.05%
C-Support Vector F1-Score: 4.55%


<br><br><br>

## LSTM

In [864]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [865]:
X = df_worthy_train["tweet_text"]
Y = df_worthy_train["class_label"]
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [867]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15)

In [872]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [877]:
def RNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(max_words, 50, input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1, name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model

In [878]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

NotImplementedError: Cannot convert a symbolic Tensor (lstm_4/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [875]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

NameError: name 'model' is not defined

<br><br><br>

## distilBERT Predictions

In [879]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

In [880]:
model_name = "bert-base-uncased"
max_length = 512

In [881]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 16.1kB/s]
Downloading: 100%|██████████| 226k/226k [00:00<00:00, 506kB/s]  
Downloading: 100%|██████████| 455k/455k [00:00<00:00, 764kB/s]  
Downloading: 100%|██████████| 570/570 [00:00<00:00, 201kB/s]


In [893]:
dataset = fetch_20newsgroups(subset="all", shuffle=True, remove=("headers","footers", "quotes"))
target_names=dataset.target_names
news_text = dataset.data
labels = dataset.target
(train_texts,valid_texts,train_labels,valid_labels)=train_test_split(news_text, labels, test_size=0.3)

from transformers import BertTokenizerFast, BertForSequenceClassification

In [891]:
news_text = df_worthy_train["tweet_text"]
labels = df_worthy_train["class_label"]
(train_texts, valid_texts, train_labels, valid_labels) = train_test_split(news_text, labels, test_size=0.3)

from transformers import BertTokenizerFast, BertForSequenceClassification

In [894]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)

Ignored unknown kwarg option direction


In [895]:
model=BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names))
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    num_train_epochs=10,
    per_device_train_batch_size=16,  # batch size per device during training
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    logging_steps=200,
    evaluation_strategy="steps",
)

Downloading: 100%|██████████| 420M/420M [01:12<00:00, 6.07MB/s] 
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenc

TypeError: __init__() missing 1 required positional argument: 'output_dir'

In [ ]:
trainer = Trainer( model=model,args=training_args,train_dataset=train_datasetataset)
trainer.train()

In [898]:
from transformers import DistilBertModel

In [899]:
from transformers import DistilBertModel, DistilBertConfig
# Initializing a DistilBERT configuration
configuration = DistilBertConfig()
# Initializing a model from the configuration
model = DistilBertModel(configuration)
# Accessing the model configuration
configuration = model.config 

In [900]:
#Similar to BERT Tokenizer, gives end-to-end tokenization for punctuation and word piece
from transformers import DistilBertTokenizer
import torch
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
inputs 

Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 11.8kB/s]
Downloading: 100%|██████████| 226k/226k [00:00<00:00, 256kB/s] 
Downloading: 100%|██████████| 455k/455k [00:00<00:00, 827kB/s]  
Downloading: 100%|██████████| 483/483 [00:00<00:00, 179kB/s]


{'input_ids': tensor([[  101,  7592,  1010,  2026,  3899,  2003, 10140,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [901]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state 

Downloading: 100%|██████████| 256M/256M [00:44<00:00, 5.99MB/s] 
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [902]:
from transformers import DistilBertTokenizer, DistilBertForMaskedLM
import torch
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForMaskedLM.from_pretrained('distilbert-base-uncased')
inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt")
labels = tokenizer("The capital of France is Paris.", return_tensors="pt")["input_ids"]
outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits 

In [904]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits 

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi